# Comparison of results

In [1]:
import os
import pickle

import numpy as np
import pandas as pd

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs, train_model
from deeppavlov.models.torch_bert.torch_transformers_classifier import TorchTransformersClassifierModel
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersPreprocessor
from joblib import load
from scipy.sparse import hstack
from scipy.sparse.csr import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import load_model

from sarcsdet.embeddings.gensim_word2vec_ruscorp import GensimWord2VecRUSEmbeddingVectorizer
from sarcsdet.embeddings.NatashaGlove import NatashaGloVeEmbeddingVectorizer
from sarcsdet.models.bilstm import BiLSTMClassifier
from sarcsdet.models.count_model_metrics import *
from sarcsdet.utils.train_utils import *

[nltk_data] Downloading package punkt to /home/kate/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kate/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kate/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


## Get data

In [2]:
with open(os.path.join('../data/Quotes', 'quotes_ling_feats.pickle'), 'rb') as f:
    df = shuffle(pickle.load(f), random_state=8)

In [3]:
_, test_df = train_test_split(df, test_size=0.3, random_state=8)

## Logistic Regression

In [4]:
tfidf = load('../data/Models/quotes/tfidf_lr/tfidf.joblib')
estimator = load('../data/Models/quotes/tfidf_lr/LogisticRegression_plain.joblib')

In [5]:
x = tfidf.transform(test_df['quote_tokenized'].values)
preds = estimator.predict_proba(x)
lr_preds = (preds[:, 1] > 0.5).astype(int)

In [6]:
lr_test_metrics = get_test_classification_metrics(
    test_df.target.values, (preds[:, 1] > 0.5).astype(int), preds[:, 1])

## BiLSTM

In [7]:
model = BiLSTMClassifier((30, 300))
model.model = load_model('../data/Models/quotes/bilstm/NatashaGlove_BiLSTM.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
embedding_model = NatashaGloVeEmbeddingVectorizer(
    False, '../data/Embeddings/navec_hudlit_v1_12B_500K_300d_100q.tar')

In [9]:
x = embedding_model.transform(test_df['quote_tokenized'])

In [10]:
preds = model.predict_proba(x)
bilstm_preds = preds

In [11]:
bilstm_test_metrics = get_test_classification_metrics(
    test_df.target.values, (preds > 0.6).astype(int), preds)

## RuBert DeepPavlov

In [12]:
bert_config = read_json(configs.classifiers.rusentiment_bert)

bert_config['dataset_reader']['x'] = 'quote'
bert_config['dataset_reader']['y'] = 'target'
bert_config['dataset_reader']['data_path'] = './'
bert_config['dataset_reader']['train'] = 'train.csv'
bert_config['dataset_reader']['valid'] = 'valid.csv'
bert_config['dataset_reader']['test'] = 'test.csv'

del bert_config['dataset_iterator']['split_seed']
del bert_config['dataset_iterator']['field_to_split']
del bert_config['dataset_iterator']['split_fields']
del bert_config['dataset_iterator']['split_proportions']

bert_config['metadata']['variables']['MODEL_PATH'] = '../data/Models/quotes/rubert/'

del bert_config['chainer']['pipe'][-2:]
del bert_config['chainer']['pipe'][1]
bert_config['chainer']['pipe'][1]['in'] = 'y'
bert_config['chainer']['pipe'][1]['depth'] = 2
bert_config['chainer']['pipe'][2]['n_classes'] = 2
bert_config['train']['metrics'] = [bert_config['train']['metrics'][-1]]
bert_config['chainer']['out'] = ['y_pred_probas']
bert_config['train']['epochs'] = 2
bert_config['train']['batch_size'] = 32
bert_config['train']['show_examples'] = True

vocab_file = '{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt'
bert_config_file = "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_config.json"
pretrained_bert = "{DOWNLOADS_PATH}/bert_models/rubert_cased_L-12_H-768_A-12_v1/bert_model.ckpt"

bert_config['chainer']['pipe'][0]['vocab_file'] = vocab_file
bert_config['chainer']['pipe'][1]['bert_config_file'] = bert_config_file
bert_config['chainer']['pipe'][1]['pretrained_bert'] = pretrained_bert

bert_config['chainer']['pipe'][2]['bert_config_file'] = bert_config_file
bert_config['chainer']['pipe'][2]['pretrained_bert'] = pretrained_bert

In [13]:
m = build_model(bert_config)











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-04-20 13:56:49.532 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/kate/Desktop/Sarcasm_Detection/data/Models/quotes/rubert/model]



INFO:tensorflow:Restoring parameters from /home/kate/Desktop/Sarcasm_Detection/data/Models/quotes/rubert/model


In [14]:
preds_proba = []
for batch in tqdm(chunks(test_df["quote"].values, 64), total=int(test_df.index.size / 64)):
    preds_proba.append(m(batch))

preds = np.concatenate(preds_proba)
rubert_preds = (preds[:, 1] > 0.5).astype(int)

In [15]:
rubert_test_metrics = get_test_classification_metrics(
    test_df.target.values, (preds[:, 1] > 0.5).astype(int), preds[:, 1])

## Quotes test metrics

In [16]:
pd.DataFrame.from_dict([
    lr_test_metrics,
    bilstm_test_metrics,
    rubert_test_metrics
]).rename(index={0: 'TFIDF_LR', 1: 'BILSTM', 2: 'RUBERT'})

,F1,Precision,Recall,PR_AUC,ROC_AUC
TFIDF_LR,0.223270,0.142651,0.513435,0.150106,0.718008
BILSTM,0.237719,0.147869,0.605856,0.163541,0.743935
RUBERT,0.257564,0.159319,0.671888,0.191431,0.778881


## Result dataframe

In [22]:
results = {
    'quote': test_df.quote.values,
    'target': test_df.target.values,
    'tfidf_lr': lr_preds,
    'bilstm': (bilstm_preds > 0.6).astype(int).flatten(),
    'rubert': rubert_preds
}

result_df = pd.DataFrame(results, columns=results.keys())
pd.set_option('display.max_colwidth', result_df.shape[0] + 1)

In [23]:
fn = result_df[
    (result_df.target == 1) & (result_df.tfidf_lr == 0) & 
    (result_df.bilstm == 0) & (result_df.rubert == 0)
]
fn.head(15)

,quote,target,tfidf_lr,bilstm,rubert
31,"Люди больше любят принимать сторону не правого, а выигравшего. Или считающегося таковым.",1,0,0,0
208,"Завелось такое количество звёздочек в кино и с такой гордыней, что не знаешь, как к ним подступиться. Ничего не сделали, глаза налево, направо скосили, но уже подавай им космический корабль.",1,0,0,0
214,— Ты стремишься продемонстрировать размеры своего своего — Двигателя? — Самолюбия!,1,0,0,0
277,Нужно подобрать что-то меньше 21 навсегда и что-то больше внезапно 42.,1,0,0,0
327,"Можно соревноваться не в ненависти, не в приобретательстве, но в любви. Это гораздо увлекательнее.",1,0,0,0
329,"Стремись достичь невозможного!, — он ведь именно так и поступает! Совершенно точно знает, что принесёт неприятности своему Ордену, и всё равно наводит смуту! Когда-нибудь он наверняка навлечёт большую беду на наш Орден!",1,0,0,0
357,"Нет ничего хуже, чем планируемое ожидание. Когда планируешь не ты.",1,0,0,0
414,"Худшее, что ты можешь сделать, Шон, — это взять ситуацию в свои руки",1,0,0,0
493,"Люди, которые должны быть забиты до смерти деревянной ложкой — это люди, которые читают книги Помоги себе сам.",1,0,0,0
711,Была волшебная погода. Дождь шел только шестьдесят дней.,1,0,0,0


In [24]:
fp = result_df[
    (result_df.target == 0) & (result_df.tfidf_lr == 1) & 
    (result_df.bilstm == 1) & (result_df.rubert == 1)
]
fp.head(15)

,quote,target,tfidf_lr,bilstm,rubert
2,"Соблюдай сабурдинацию, пожалуйста.",0,1,1,1
9,"Нация меряется по гражданству. В данном случае все россияне. За границей мы и сейчас все русские — будь ты чеченец, татарин или удмурт. Нация по гражданству, пожалуй, единственно верный подход. Американец может быть по происхождению мексиканцем, итальянцем, темнокожим, но он все равно американец.",0,1,1,1
12,"Остерегайся того, кто прикидывается хромым.",0,1,1,1
18,"Эта гидра, это чудовище — самоуправство административных властей, развращенность чиновничества и подкупность судов России. Россия стонет в тисках этой гидры поборов, насилия и грабежа, которая до сих пор издевается даже над высшей властью.",0,1,1,1
28,"— Какой же я был дурак, что усомнился в тебе! Доктор Пиндер-Шлосс все объяснила. — Очаровательная женщина, такая зловещая.",0,1,1,1
45,"Там, откуда я родом, мы считаем религию непристойно скучной.",0,1,1,1
51,"— Гриша, стой! Нам нужна мораль! — Нужна мораль — позвони маме.",0,1,1,1
75,"Вольный воздух чаще всего бывает лишь в закрытых помещениях. Интересно, что об этом могла бы сказать физика?",0,1,1,1
85,"— Или вот, — сказал Борис Григорьевич, — в Америке около тысячи женщин беременны от инопланетян. У нас тоже таких полно, но их КГБ где-то прячет.",0,1,1,1
86,"— Я больше не могу! Я скоро повешусь! — Ты ноги натёр. Траву приложи. — Да меня не мозоли достали, а вон та принцесса-трещотка!",0,1,1,1


In [25]:
with open('../results/quote_fn.txt', 'w') as f:
    f.write('\n'.join(fn.quote.to_list()))

In [26]:
with open('../results/quote_fp.txt', 'w') as f:
    f.write('\n'.join(fp.quote.to_list()))